In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !cp drive/MyDrive/ssne/train.pkl train.pkl
# !cp drive/MyDrive/ssne/test_no_target.pkl test_no_target.pkl

In [3]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.functional import one_hot

In [4]:
with open('train.pkl', 'rb') as file:
  data = pickle.load(file)

In [5]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [6]:
class ClassicalMusicDataset(Dataset):
  def __init__(self, features, targets):
    self.features = [torch.Tensor(f).float() for f in features]
    self.targets = targets

  def __getitem__(self, idx):
    return self.features[idx], self.targets[idx]

  def __len__(self):
    return len(self.features)


In [7]:
 classes = {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}

In [8]:
features, labels = [], []
for feature, label in data:
  features.append(feature)
  labels.append(label)

features, labels = np.array(features, dtype=object), np.array(labels)

In [9]:
TEST_TRAIN_RATIO = 0.2

train_indices = np.random.rand(len(features)) > TEST_TRAIN_RATIO
test_indices = ~train_indices

train_set = ClassicalMusicDataset(features[train_indices], labels[train_indices])
test_set = ClassicalMusicDataset(features[test_indices], labels[test_indices])

In [10]:
def pad_collate(batch):
  (xx, yy) = zip(*batch)
  x_lens = [len(x) for x in xx]

  xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

  return xx_pad, yy, x_lens

In [11]:
data_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True, collate_fn=pad_collate, drop_last=True)
test_data_loader = DataLoader(dataset=train_set, batch_size=len(test_set), collate_fn=pad_collate)

In [12]:
class RNNRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.rnn = nn.GRU(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden
    
    def forward(self, x, x_len, hidden):
        packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        all_outputs, hidden = self.rnn(packed, hidden)
        all_outputs, _ = pad_packed_sequence(all_outputs, batch_first=True, padding_value=0)
        out = all_outputs[:,-1,:] # We are interested only on the last output
        x = self.fc(out.float())
        x = self.softmax(x)
        return x, hidden

In [13]:
model = RNNRegressor(1, 64, 2, len(classes)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.MSELoss()

model.train()
for epoch in range(1):
  for x,targets,x_len in data_loader:
    x = x.unsqueeze(2).to(device)
    targets = torch.LongTensor(targets).to(device)
    targets = one_hot(targets, len(classes))
    hidden = model.init_hidden(64)
    hidden = hidden.to(device)
    preds, _ = model(x,x_len, hidden)
    optimizer.zero_grad()
    loss = loss_fun(preds, targets.float())
    loss.backward()
    optimizer.step()
  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, loss: {loss.item():.3}")

C:\Users\kubar\AppData\Local\Temp/ipykernel_25564/2162881845.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Epoch: 0, loss: 0.16


In [14]:
model.eval()
for x, targets, x_len in test_data_loader:
    x = x.unsqueeze(2).to(device)
    targets = torch.LongTensor(targets).to(device)
    targets = one_hot(targets, len(classes))
    hidden = model.init_hidden(64)
    hidden = hidden.to(device)
    preds, _ = model(x,x_len, hidden)

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED